#Maybe overkill...

In [ ]:
import random as rd
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

#submit = True
submit = False

In [ ]:
# load the public MNIST dataset.
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000,784)
x_train_norm = x_train/255.

x_test = x_test.reshape(10000,784)
x_test_norm = x_test/255.
x_test_norm = x_test.reshape((10000, 28, 28, 1))

In [ ]:
#k_train = pd.read_csv('../input/digit-recognizer/train.csv')
#k_labels = np.asarray(k_train['label'])
#k_train = k_train.drop(columns=['label'])
#k_train_norm = np.asarray(k_train/255.)

#val_set_labels = k_labels[-2000:]
#val_set_norm = k_train_norm[-2000:]
#val_set_norm = val_set_norm.reshape(2000, 28, 28, 1)

#k_train_norm = k_train_norm[:-2000]
#k_labels = k_labels[:-2000]

In [ ]:
#x_data = [row for row in x_test_norm]
x_data = [row for row in x_train_norm]
#x_data += [row for row in k_train_norm]
x_data = np.asarray(x_data)
x_data = x_data.reshape((60000, 28, 28, 1))

#y_data = [label for label in y_test]
y_data = [label for label in y_train]
#y_data += [label for label in k_labels]
y_data = np.asarray(y_data).flatten()

In [ ]:
# train on the training set with some held back for validation #
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=None, verbose=0):

    history = model.fit(x=train_features, y=train_label, 
                        batch_size=batch_size,
                        epochs=epochs, shuffle=True, 
                        validation_split=validation_split,
                        verbose = verbose)

    # Gather the model's metrics after each round of training
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist

In [ ]:
 # Set up a CNN with Keras
 
def create_CNN(learning_rate):
    """Create and compile a convolutional neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 5, activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Conv2D(64, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [ ]:
def create_mixedNN(learning_rate):
    """Create and compile a deep neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, 6, activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, 2, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model

In [ ]:
model_C = create_mixedNN(learning_rate)
model_C.summary()

In [ ]:
model_C.layers[-3].output

In [ ]:
def getDNNModel(learning_rate):
    inputs = keras.Input(shape=(28, 28, 1))
    dnn = layers.Flatten()(inputs)
    dnn = layers.Dense(768, activation='relu')(dnn)
    dnn = layers.Dropout(0.3)(dnn)
    dnn = layers.Dense(units=512, activation='relu')(dnn)
    dnn = layers.Dropout(0.2)(dnn)
    dnn = layers.Dense(units=256, activation='relu')(dnn)
    dnn = layers.Dropout(0.1)(dnn)
    outputs = layers.Dense(10, activation="softmax")(dnn)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model    

In [ ]:
# Train and evalate CNN on ALL the datasets, 
# Train on 10000 + 60000 + 42000 to predict on 28000
learning_rate = 0.001
epochs = 50
batch_size = 300

if submit:
    validation_split = None
    verbose = 0
else:
    validation_split = 0.1 # off bottom
    verbose = 1
start_time = time.time()

model_A = create_CNN(learning_rate)
model_B = getDNNModel(learning_rate)
model_C = create_mixedNN(learning_rate)

#x_test_norm:
epochs_A, hist_A = train_model(model_A, x_test_norm, y_test, epochs, batch_size, validation_split, verbose=verbose)
if not submit:
    print("\nTrained model A in about {} seconds\n".format((round(time.time()-start_time,4))))
batch_time = time.time()

#x_train_norm:
epochs_B, hist_B = train_model(model_B, x_test_norm, y_test, epochs, batch_size, validation_split, verbose=verbose)
if not submit:
    print("\nTrained model B in about {} seconds\n".format((round(time.time()-start_time,4))))
batch_time = time.time()

#k_train_norm:
epochs_C, hist_C = train_model(model_C, x_test_norm, y_test, epochs, batch_size, validation_split, verbose=verbose)
if not submit:
    print("\nTrained model C in about {} seconds\n".format((round(time.time()-start_time,4))))

if not submit:
    print ("\nTotal time: {} seconds is about {} minutes.".format(round(time.time()-start_time,4),
                                                              (time.time()-start_time)//60))

In [ ]:
model_A.save('model_A1')
model_B.save('model_B1')
model_C.save('model_C1')

In [ ]:
if not submit:
    print ("Evaluate against x_test_norm:")
    print ("A:",model_A.evaluate(x_test_norm, y_test, batch_size=100))
    print ("B:",model_B.evaluate(x_test_norm, y_test, batch_size=100))
    print ("C:",model_C.evaluate(x_test_norm, y_test, batch_size=100))

In [ ]:
def getUberModel(learning_rate, modelA, modelB, modelC):
    
    inputs = keras.Input(shape=(28, 28, 1))
    xA = modelA(inputs)
    xB = modelB(inputs)
    xC = modelC(inputs)
    x = layers.concatenate([xA, xB, xC])
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    ensemble_model = keras.Model(inputs=inputs, outputs=outputs)
    ensemble_model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                            loss="sparse_categorical_crossentropy",
                            metrics=['accuracy']) 
    return ensemble_model

In [ ]:
ubermodel = getUberModel(learning_rate, model_A, model_B, model_C)
ubermodel.summary()

In [ ]:
keras.utils.plot_model(ubermodel, "ubermodel.png")

In [ ]:
uberdataA = getProbabilities(model_A, x_test_norm)
uberdataB = getProbabilities(model_B, x_test_norm)
uberdataC = getProbabilities(model_C, x_test_norm)
uberdata = np.zeros((10000,30))
uberdata[:,:10] = uberdataA
uberdata[:,10:20] = uberdataB
uberdata[:,20:30] = uberdataC

uberdata = uberdata.reshape(-1,30,1)

In [ ]:
#Ubermodel train:

epochs_U, hist_U = train_model(ubermodel, x_test_norm, y_test, epochs, batch_size, validation_split, verbose=verbose)

if not submit:
    print("\nTrained model C in about {} seconds\n".format((round(time.time()-start_time,4))))


In [ ]:
def getPredictions(model=None, dataset=None): 
    ''' Takes a trained model and data
        Returns a List of guesses for that model on that data
    ''' 
    predictions = []
    predicts = model.predict(dataset)
    for j in range(len(dataset)):
        probs = predicts[j] # one row of 10 probabilities 
        max_id = np.argmax(probs)   # index of top probability in row
        predictions += [max_id]
    return predictions

In [ ]:
def getProbabilities(model=None, dataset=None):
    ''' Takes a trained model and a set of data
        Returns a List of guesses by that model for that data
    ''' 
    return model.predict(dataset)

In [ ]:
kaggle = pd.read_csv('../input/digit-recognizer/test.csv')
kaggle_norm = np.asarray(kaggle/255.)
kaggle_norm = kaggle_norm.reshape(28000, 28, 28, 1)

In [ ]:
kag_guesses = pd.DataFrame(columns=['A', 'B', 'C', 'AB','AC','CB'])

kag_guesses['A'] = getPredictions(model=model3D_A, dataset=kaggle_norm)
kag_guesses['B'] = getPredictions(model=model3D_B, dataset=kaggle_norm)
kag_guesses['C'] = getPredictions(model=model3D_C, dataset=kaggle_norm)

for i in range(len(kag_guesses['A'])):
    kag_guesses['AB'] = kag_guesses['A'] - kag_guesses['B']
    kag_guesses['AC'] = kag_guesses['A'] - kag_guesses['C']
    kag_guesses['CB'] = kag_guesses['C'] - kag_guesses['B']

In [ ]:
kag_probs = pd.DataFrame(columns=list(range(30)))

getProbabilities()
uberguesses = getPredictions(model=ubermodel, )

In [ ]:
if not submit:
#All agree on them all!?
    kag_guesses['CB'].sum() + kag_guesses['AC'].sum() + kag_guesses['AB'].sum()
    kag_guesses.to_csv('guessesABC.csv')

In [ ]:
if submit:
    kaggles = pd.DataFrame(columns=['ImageId','Label']) 
    kaggles['Label'] = kag_guesses['C']
    kaggles['ImageId'] = [i+1 for i in kaggles.index.values] 
    kaggles.to_csv('submission.csv', columns=["ImageId","Label"], index=False)

##Visualizations

In [ ]:
k_img = np.asarray(kaggle.iloc[20044 - 1])
k_img = k_img.reshape(28,28)
plt.imshow(k_img)

In [ ]:
if not submit:
# Plot a graph of the 'accuracy' metric vs. epochs:
    plt.plot(epochs_A,hist_A["accuracy"])
    plt.plot(epochs_A,hist_A["val_accuracy"])
    plt.show()

    plt.plot(epochs_B,hist_B["accuracy"])
    plt.plot(epochs_B,hist_B["val_accuracy"])
    plt.show()